In [3]:
%%time
# endpoint invocation script
# expects the file test_point.csv to be located in the same diretory

#endpoint_name = 'XGBoostEP-2020-03-15-16-23-35'
#endpoint_name = 'XGBoostEP-2020-03-15-20-37-40'
endpoint_name = 'XGBoostEP-2020-03-19-01-50-07'

import os
import boto3
import re
import json
from sagemaker import get_execution_role
import sagemaker
import time

sess = sagemaker.Session()
region = boto3.Session().region_name
role = get_execution_role()
bucket=sess.default_bucket()

runtime_client = boto3.client('runtime.sagemaker')

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
#print('Predicted Class Probabilities: {}.'.format(result))
print(response)

file_name = 'test_point.csv'
with open(file_name, 'r') as f:
    payload = f.read().strip()
    

{'ResponseMetadata': {'RequestId': 'c3cba323-4b9d-4490-a9a6-77785efdd1da', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c3cba323-4b9d-4490-a9a6-77785efdd1da', 'x-amzn-invoked-production-variant': 'model-Blue', 'date': 'Thu, 19 Mar 2020 01:58:22 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '2'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'model-Blue', 'Body': <botocore.response.StreamingBody object at 0x7f5957f22198>}
CPU times: user 508 ms, sys: 42.8 ms, total: 550 ms
Wall time: 873 ms


In [4]:
N = 20000 #5000

for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    if i % (N/25) == 0:
        invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
        invocation_code = response['ResponseMetadata']['HTTPStatusCode']
        print ("target model: {}, return code {}; {} of {} invocations".format(invoked_model, invocation_code, i, N))
    #time.sleep(0.1)

target model: model-Blue, return code 200; 0 of 20000 invocations
target model: model-Blue, return code 200; 200 of 20000 invocations
target model: model-Blue, return code 200; 400 of 20000 invocations
target model: model-Blue, return code 200; 600 of 20000 invocations
target model: model-Blue, return code 200; 800 of 20000 invocations
target model: model-Blue, return code 200; 1000 of 20000 invocations
target model: model-Blue, return code 200; 1200 of 20000 invocations
target model: model-Blue, return code 200; 1400 of 20000 invocations
target model: model-Blue, return code 200; 1600 of 20000 invocations
target model: model-Blue, return code 200; 1800 of 20000 invocations
target model: model-Blue, return code 200; 2000 of 20000 invocations
target model: model-Blue, return code 200; 2200 of 20000 invocations
target model: model-Blue, return code 200; 2400 of 20000 invocations
target model: model-Blue, return code 200; 2600 of 20000 invocations
target model: model-Blue, return code 200

KeyboardInterrupt: 

In [ ]:
N = 200 #5000
print("started model invocation for {} iterations". format(N))
for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
    invocation_code = response['ResponseMetadata']['HTTPStatusCode']
    if (invocation_code != 200):
        print ("error invoking model {} ".format(invocation_code))